In [1]:
# https://www.kaggle.com/mathormad/inceptionv3-baseline-lb-0-379/code

In [2]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image
import cv2
from sklearn.utils import class_weight, shuffle

import warnings
warnings.filterwarnings("ignore")
SIZE = 512

In [3]:
# https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric/notebook
import tensorflow as tf

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1) + (1 - K.mean(f1))

# POS_WEIGHT = 1.0  # multiplier for positive targets, needs to be tuned

# def weighted_binary_crossentropy(target, output):
#     """
#     Weighted binary crossentropy between an output tensor 
#     and a target tensor. POS_WEIGHT is used as a multiplier 
#     for the positive targets.

#     Combination of the following functions:
#     * keras.losses.binary_crossentropy
#     * keras.backend.tensorflow_backend.binary_crossentropy
#     * tf.nn.weighted_cross_entropy_with_logits
#     """
#     # transform back to logits
# #     _epsilon = tfb._to_tensor(tfb.epsilon(), output.dtype.base_dtype)
#     _epsilon = K.epsilon()
#     output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
#     output = tf.log(output / (1 - output))
#     # compute weighted loss
#     loss = tf.nn.weighted_cross_entropy_with_logits(targets=target,
#                                                     logits=output,
#                                                     pos_weight=POS_WEIGHT)
#     return tf.reduce_mean(loss, axis=-1)

# import tensorflow as tf
# from tensorflow.python.framework import ops
# from functools import reduce

# def binaryRound(x):
#     """
#     Rounds a tensor whose values are in [0,1] to a tensor with values in {0, 1},
#     using the straight through estimator for the gradient.
#     """
#     g = tf.get_default_graph()

#     with ops.name_scope("BinaryRound") as name:
#         with g.gradient_override_map({"Round": "Identity"}):
#             return tf.round(x, name=name)

#         # For Tensorflow v0.11 and below use:
#         #with g.gradient_override_map({"Floor": "Identity"}):
#         #    return tf.round(x, name=name)
        
# def f1_loss2(y_true, y_pred):
#     y_pred = binaryRound(y_pred)
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
#     tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
#     fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
#     fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

#     p = tp / (tp + fp + K.epsilon())
#     r = tp / (tp + fn + K.epsilon())

#     f1 = 2*p*r / (p+r+K.epsilon())
#     f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
#     return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1) + (1-K.mean(f1))

In [4]:
# Load dataset info
path_to_train = '../data/train/'
data = pd.read_csv('../data/train.csv')

In [5]:
data.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [6]:
train_dataset_info = []
for name, labels in zip(data['Id'], data['Target'].str.split(' ')):
    train_dataset_info.append({
        'path':os.path.join(path_to_train, name),
        'labels':np.array([int(label) for label in labels])})
train_dataset_info = np.array(train_dataset_info)

In [7]:
train_dataset_info

array([{'path': '../data/train/00070df0-bbc3-11e8-b2bc-ac1f6b6435d0', 'labels': array([16,  0])},
       {'path': '../data/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0', 'labels': array([7, 1, 2, 0])},
       {'path': '../data/train/000a9596-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([5])},
       ...,
       {'path': '../data/train/fff189d8-bbab-11e8-b2ba-ac1f6b6435d0', 'labels': array([7])},
       {'path': '../data/train/fffdf7e0-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([25,  2, 21])},
       {'path': '../data/train/fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0', 'labels': array([2, 0])}],
      dtype=object)

In [8]:
submit = pd.read_csv('../data/sample_submission.csv')
path_to_test = '../data/test/'
test_dataset_info = []
for name in tqdm(submit['Id']):
    test_dataset_info.append({
        'path':os.path.join(path_to_test, name)})
test_dataset_info = np.array(test_dataset_info)

100%|██████████| 11702/11702 [00:00<00:00, 77941.80it/s]


In [9]:
# rgb_arr = np.memmap('../cache/tmp_rgb_arr', dtype='uint8', mode='r+', 
#                    shape=(len(train_dataset_info),299,299,3))

In [10]:
class data_generator:
    
    def create_train(dataset_info, batch_size, shape, augument=True):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_train_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_train_batch), 28))
                for i in range(len(X_train_batch)):
                    image = data_generator.load_image(
                        X_train_batch[i]['path'], shape) 
#                     image = data_generator.load_image(
#                         i, shape) 
                    if augument:
                        image = data_generator.augment(image)
                    batch_images.append(image/255.)
                    batch_labels[i][X_train_batch[i]['labels']] = 1
                yield np.array(batch_images, np.float32), [batch_labels, np.array(batch_images, np.float32)]
    def create_train2(dataset_info, batch_size, shape, augument=False):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_train_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_train_batch), 28))
                for i in range(len(X_train_batch)):
                    image = data_generator.load_image(
                        X_train_batch[i]['path'], shape) 
#                     image = data_generator.load_image(
#                         i, shape) 
                    
                    batch_images.append(image/255.)
                    
                yield np.array(batch_images, np.float32), None
    def create_test(dataset_info, batch_size, shape, augument=False):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_test_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_test_batch), 28))
                for i in range(len(X_test_batch)):
                    image = data_generator.load_image(
                        X_test_batch[i]['path'], shape) 
#                     image = data_generator.load_image(
#                         i, shape) 
                    
                    batch_images.append(image/255.)
                    
                yield np.array(batch_images, np.float32), None
    def load_image(path, shape):
        img1 = cv2.imread(path+'_red.png', cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(path+'_green.png', cv2.IMREAD_GRAYSCALE)
        img3 = cv2.imread(path+'_blue.png', cv2.IMREAD_GRAYSCALE)
        image = np.stack((img1,img2,img3), -1)
        image = cv2.resize(image, (shape[0], shape[1]))
        return image
    def load_image3(idx, shape):
#         print(idx)
        name = '../cache/RGB/img-{}.png'.format(idx)
        image = cv2.imread(name)
        image = cv2.resize(image, (shape[0], shape[1]))
        return image
    def load_image2(path, shape):
        image_red_ch = Image.open(path+'_red.png')
        image_yellow_ch = Image.open(path+'_yellow.png')
        image_green_ch = Image.open(path+'_green.png')
        image_blue_ch = Image.open(path+'_blue.png')
        image = np.stack((
            np.array(image_red_ch),
            np.array(image_green_ch), 
            np.array(image_blue_ch)), -1)
        w, h = 512, 512
#         zero_data = np.zeros((h, w), dtype=np.uint8)
#         image2 = np.stack((
#             np.array(image_yellow_ch),
#             zero_data, zero_data), -1)
#         print(image1.shape, image2.shape)
#         image = np.vstack((image1, image2))
        image = cv2.resize(image, (shape[0], shape[1]))
        return image
    
    def augment2(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
            ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug
    def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                    iaa.Fliplr(0.5), # horizontal flips
                    iaa.Crop(percent=(0, 0.1)), # random crops
                    # Small gaussian blur with random sigma between 0 and 0.5.
                    # But we only blur about 50% of all images.
                    iaa.Sometimes(0.5,
                        iaa.GaussianBlur(sigma=(0, 0.5))
                    ),
                    # Strengthen or weaken the contrast in each image.
                    iaa.ContrastNormalization((0.75, 1.5)),
                    # Add gaussian noise.
                    # For 50% of all images, we sample the noise once per pixel.
                    # For the other 50% of all images, we sample the noise per pixel AND
                    # channel. This can change the color (not only brightness) of the
                    # pixels.
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                    # Make some images brighter and some darker.
                    # In 20% of all cases, we sample the multiplier once per channel,
                    # which can end up changing the color of the images.
                    iaa.Multiply((0.8, 1.2), per_channel=0.2),
                    # Apply affine transformations to each image.
                    # Scale/zoom them, translate/move them, rotate them and shear them.
                    iaa.Affine(
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                        rotate=(-180, 180),
                        shear=(-8, 8)
                    )
                ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug
    

In [11]:
import keras as k
print(k.__version__)

Using TensorFlow backend.


2.2.4


In [12]:
# !pip install -U keras

In [13]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D
from keras.layers import UpSampling2D, Lambda, Reshape, Conv2DTranspose
from keras.layers import  MaxPooling2D, Concatenate, ReLU, LeakyReLU, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model

In [14]:
# network parameters
input_shape = (512, )
intermediate_dim = 128
batch_size = 16
latent_dim = 16
epochs = 3

In [15]:
kernel_size = 3
layer_filters = [32, 64, 128, 256, 512]

# Build the Autoencoder Model
# First build the Encoder Model
def encoder(input_shape):
    inputs = Input(shape=input_shape, name='encoder_input')
    x = inputs
    # Stack of Conv2D blocks
    # Notes:
    # 1) Use Batch Normalization before ReLU on deep networks
    # 2) Use MaxPooling2D as alternative to strides>1
    # - faster but not as good as strides>1
    for filters in layer_filters:
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=2,
                   activation='relu',
                   padding='same')(x)

    shape = K.int_shape(x)

    # Generate the latent vector
    x = Flatten()(x)
    latent = Dense(latent_dim, name='latent_vector')(x)
    print(latent.shape)
    return latent, shape

def decoder(shape):
    latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
    x = Dense(shape[1] * shape[2] * shape[3])(latent_inputs)
    x = Reshape((shape[1], shape[2], shape[3]))(x)

    # Stack of Transposed Conv2D blocks
    # Notes:
    # 1) Use Batch Normalization before ReLU on deep networks
    # 2) Use UpSampling2D as alternative to strides>1
    # - faster but not as good as strides>1
    for filters in layer_filters[::-1]:
        x = Conv2DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            strides=2,
                            activation='relu',
                            padding='same')(x)

    x = Conv2DTranspose(filters=1,
                        kernel_size=kernel_size,
                        padding='same')(x)

    outputs = Activation('sigmoid', name='decoder_output')(x)
    
    return output

In [16]:
l_train = len(train_dataset_info) 
l_test = 11702

In [17]:
len_all = l_train + l_test
len_all

42774

In [18]:
def global_average_pooling(x):
    return K.mean(x, axis = (2, 3))
    
def global_average_pooling_shape(input_shape):
    return input_shape[0:2]

In [19]:
def create_model(input_shape, n_out=28):
    
    inputs = Input(shape=input_shape, name='encoder_input')
    x = inputs
    # Stack of Conv2D blocks
    # Notes:
    # 1) Use Batch Normalization before ReLU on deep networks
    # 2) Use MaxPooling2D as alternative to strides>1
    # - faster but not as good as strides>1
    for filters in layer_filters:
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=2,
                   activation='relu',
                   padding='same')(x)

    shape = K.int_shape(x)

    # Generate the latent vector
    x = Flatten()(x)
    latent_inputs = Dense(latent_dim, name='latent_vector')(x)
    
#     latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
    x = Dense(shape[1] * shape[2] * shape[3])(latent_inputs)
    x = Reshape((shape[1], shape[2], shape[3]))(x)

    # Stack of Transposed Conv2D blocks
    # Notes:
    # 1) Use Batch Normalization before ReLU on deep networks
    # 2) Use UpSampling2D as alternative to strides>1
    # - faster but not as good as strides>1
    for filters in layer_filters[::-1]:
        x = Conv2DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            strides=2,
                            activation='relu',
                            padding='same')(x)

    x = Conv2DTranspose(filters=3,
                        kernel_size=kernel_size,
                        padding='same')(x)

    img_out = Activation('sigmoid', name='decoder_output')(x)
   
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    img_out = Dense(3, activation='relu',name='img_out')(img_out)
    output = Dense(n_out, activation='sigmoid', name='output')(x)
    model = Model(inputs, [output, img_out])
    
    return model

In [20]:
model = create_model(
    input_shape=(SIZE,SIZE,3))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 896         encoder_input[0][0]              
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 64) 18496       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 64, 64, 128)  73856       conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (

In [21]:
def create_model1(input_shape, n_out=28):
    input_tensor = Input(shape=input_shape)
    base_model = InceptionV3(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape)
    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    x = Lambda(global_average_pooling, output_shape=global_average_pooling_shape)(x)
    
#     x = Conv2D(256, kernel_size=(1,1), activation='relu', name='conv2')(x)
#     x = Conv2D(32, kernel_size=(1,1), activation='relu', name='conv1')(x)
#     x = Flatten()(x)
#     x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    z_mean = Dense(latent_dim)(x)
    z_log_var = Dense(latent_dim)(x)
    z = Lambda(sampling)([z_mean, z_log_var])
#     z = Reshape((1,latent_dim))(z)
    img_out = Lambda(decode_sample)(z)
    img_out = Dense(3, activation='relu')(img_out)
    output = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, [output, img_out])
    
    return model

In [22]:
# some basic useless model
def create_mode2l(input_shape):
    
    dropRate = 0.5
    
    init = Input(input_shape)
    x = BatchNormalization(axis=-1)(init)
    x = Conv2D(32, (3, 3))(x) #, strides=(2,2))(x)
    x = ReLU()(x)

    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    ginp1 = Dropout(dropRate)(x)
    
    x = BatchNormalization(axis=-1)(ginp1)
    x = Conv2D(64, (3, 3), strides=(2,2))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(64, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(64, (3, 3))(x)
    x = ReLU()(x)
    
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    ginp2 = Dropout(dropRate)(x)
    
    x = BatchNormalization(axis=-1)(ginp2)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    ginp3 = Dropout(dropRate)(x)
    
    gap1 = GlobalAveragePooling2D()(ginp1)
    gap2 = GlobalAveragePooling2D()(ginp2)
    gap3 = GlobalAveragePooling2D()(ginp3)
    
    x = Concatenate()([gap1, gap2, gap3])
    
    x = BatchNormalization(axis=-1)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropRate)(x)
    
    x = BatchNormalization(axis=-1)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    x = Dense(28)(x)
    x = Activation('sigmoid')(x)
    
    model = Model(init, x)
    
    return model

In [23]:
# create callbacks list
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

epochs = 10; batch_size = 16
checkpoint = ModelCheckpoint('../cache/ae36.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, 
                                   verbose=1, mode='auto', epsilon=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=6)
callbacks_list = [checkpoint, early, reduceLROnPlat]


In [24]:

# split data into train, valid
indexes = np.arange(train_dataset_info.shape[0])
np.random.shuffle(indexes)
train_indexes, valid_indexes = train_test_split(indexes, test_size=0.15, random_state=8)




In [25]:
# warm up model
# model = create_model(
#     input_shape=(SIZE,SIZE,3), 
#     n_out=28)
model.summary()

# for layer in model.layers:
#     layer.trainable = False
# model.layers[-1].trainable = True
# model.layers[-2].trainable = True
# model.layers[-3].trainable = True
# model.layers[-4].trainable = True
# model.layers[-5].trainable = True
# model.layers[-6].trainable = True
# model.layers[-7].trainable = True
# model.layers[-8].trainable = True

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 896         encoder_input[0][0]              
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 64) 18496       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 64, 64, 128)  73856       conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (

In [26]:
# model.compile(
#     loss=[f1_loss], 
#     optimizer=Adam(1e-03),
#     metrics=[f1])


In [27]:
print(len(train_indexes), len(valid_indexes))

26411 4661


In [28]:
# batch_size=8

# # create train and valid datagens
# train_generator = data_generator.create_train(
#     train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
# validation_generator = data_generator.create_train(
#     train_dataset_info[valid_indexes], batch_size, (SIZE,SIZE,3), augument=False)

# model.fit_generator(
#     train_generator,
#     steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
#     validation_data=validation_generator,
#     validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
#     epochs=2, 
#     verbose=1)

In [29]:
# for layer in model.layers:
#     layer.trainable = False
# model.layers[-1].trainable = True
# model.layers[-2].trainable = True
# model.layers[-3].trainable = True
# model.layers[-4].trainable = True
# model.layers[-5].trainable = True
# model.layers[-6].trainable = True
# model.layers[-7].trainable = True
# model.layers[-8].trainable = True
# model.layers[-9].trainable = True

In [30]:
# model.compile(
#     loss=[f1_loss, 'mse'], 
#     optimizer=Adam(1e-04),
#     metrics=[f1])

In [31]:
# batch_size=16

# train_generator = data_generator.create_train(
#     train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
# validation_generator = data_generator.create_train(
#     train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=False)

# model.fit_generator(
#     train_generator,
#     steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
#     validation_data=validation_generator,
#     validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
#     epochs=2, 
#     verbose=1,
#     callbacks=callbacks_list)

In [32]:
epochs=120

batch_size=8
for layer in model.layers:
    layer.trainable = True
model.compile(loss=[f1_loss, 'mse'],
              optimizer=Adam(lr=1e-4), 
              loss_weights = {"output": 1.0, "img_out": 10.0 }, 
              metrics=[f1])

In [33]:
# train all layers
batch_size=8

train_generator = data_generator.create_train(
    train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
validation_generator = data_generator.create_train(
    train_dataset_info[valid_indexes], 16, (SIZE,SIZE,3), augument=False)

model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=epochs, 
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/120
3302/3302 [==============================] - 1143s 346ms/step - loss: 2.3519 - output_loss: 1.1481 - img_out_loss: 0.1204 - output_f1: 0.0356 - img_out_f1: 0.0299 - val_loss: 1.4612 - val_output_loss: 1.0975 - val_img_out_loss: 0.0364 - val_output_f1: 0.0444 - val_img_out_f1: 0.0000e+00

Epoch 00001: val_loss improved from inf to 1.46125, saving model to ../cache/ae36.h5
Epoch 2/120
3302/3302 [==============================] - 1126s 341ms/step - loss: 1.3366 - output_loss: 1.1056 - img_out_loss: 0.0231 - output_f1: 0.0529 - img_out_f1: 0.0000e+00 - val_loss: 1.2711 - val_output_loss: 1.0649 - val_img_out_loss: 0.0206 - val_output_f1: 0.0730 - val_img_out_f1: 0.0000e+00

Epoch 00002: val_loss improved from 1.46125 to 1.27113, saving model to ../cache/ae36.h5
Epoch 3/120
3302/3302 [==============================] - 1121s 340ms/step - loss: 1.2885 - output_loss: 1.0870 - img_out_loss: 0.0201 - output_f1: 0.0682 - img_out_f1: 0.0000e+00 - val_loss: 1.2526 - val_output_loss: 1.0

KeyboardInterrupt: 

In [53]:
# for ii in tqdm(np.arange(len(train_dataset_info))):
#     img1 = cv2.imread(train_dataset_info[ii]['path']+'_red.png', cv2.IMREAD_GRAYSCALE)
#     img2 = cv2.imread(train_dataset_info[ii]['path']+'_green.png', cv2.IMREAD_GRAYSCALE)
#     img3 = cv2.imread(train_dataset_info[ii]['path']+'_blue.png', cv2.IMREAD_GRAYSCALE)
#     img1 = np.stack((img1, img2, img3), -1)
#     name = '../cache/RGB/img-{}.png'.format(ii)
#     cv2.imwrite(name,img1)

In [54]:
# submit = pd.read_csv('../data/sample_submission.csv')
# for name in tqdm(submit['Id']):
#     path = os.path.join('../data/test/', name)
#     img1 = cv2.imread(path+'_red.png', cv2.IMREAD_GRAYSCALE)
#     img2 = cv2.imread(path+'_green.png', cv2.IMREAD_GRAYSCALE)
#     img3 = cv2.imread(path+'_blue.png', cv2.IMREAD_GRAYSCALE)
#     img1 = np.stack((img1, img2, img3), -1)
#     name1 = '../cache/RGB/test/img-{}.png'.format(name)
#     cv2.imwrite(name1,img1)

In [120]:
# Create submit
submit = pd.read_csv('../data/sample_submission.csv')
predicted = []
draw_predict = []
# model = create_model(
#     input_shape=(SIZE,SIZE,3), 
#     n_out=28)
# for layer in model.layers:
#     layer.trainable = True
# model.compile(loss=f1_loss,
#             optimizer=Adam(lr=1e-4),
#             metrics=[f1])
model.load_weights('../cache/ae36.h5')
for name in tqdm(submit['Id']):
    path = os.path.join('../data/test/', name)
    image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
    score_predict= model.predict(image[np.newaxis])[0]
    
    draw_predict.append(score_predict[0])
    label_predict = np.arange(28)[score_predict[0]>=0.2]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

submit['Predicted'] = predicted
# np.save('../cache/draw_predict_InceptionV3-8.npy', score_predict)
# submit.to_csv('../submissions/submit_InceptionV3.csv', index=False)

100%|██████████| 11702/11702 [14:00<00:00, 13.92it/s]


In [121]:
submit.to_csv('../submissions/sub36-a.csv', index=False)

In [61]:
predicted = []
# for name in tqdm(submit['Id']):
for score_predict in tqdm(draw_predict):
#     path = os.path.join('../data/test/', name)
#     image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
#     score_predict = model.predict(image[np.newaxis])[0]
#     draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict>=0.25]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit['Predicted'] = predicted
submit.to_csv('../submissions/sub35b-b.csv', index=False)

100%|██████████| 11702/11702 [00:00<00:00, 49615.81it/s]


In [62]:
predicted = []
# for name in tqdm(submit['Id']):
for score_predict in tqdm(draw_predict):
#     path = os.path.join('../data/test/', name)
#     image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
#     score_predict = model.predict(image[np.newaxis])[0]
#     draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict>=0.3]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit['Predicted'] = predicted
submit.to_csv('../submissions/sub35b-c.csv', index=False)

100%|██████████| 11702/11702 [00:00<00:00, 50116.60it/s]


In [63]:
predicted = []
# for name in tqdm(submit['Id']):
for score_predict in tqdm(draw_predict):
#     path = os.path.join('../data/test/', name)
#     image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
#     score_predict = model.predict(image[np.newaxis])[0]
#     draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict>=0.35]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit['Predicted'] = predicted
submit.to_csv('../submissions/sub35b-d.csv', index=False)

100%|██████████| 11702/11702 [00:00<00:00, 53176.84it/s]


In [32]:
#https://stackoverflow.com/questions/1855095/how-to-create-a-zip-archive-of-a-directory
def backup_project_as_zip(project_dir, zip_file):
    assert(os.path.isdir(project_dir))
    assert(os.path.isdir(os.path.dirname(zip_file)))
    shutil.make_archive(zip_file.replace('.zip',''), 'zip', project_dir)
    pass

In [33]:
import datetime, shutil
now = datetime.datetime.now()
print(now)
PROJECT_PATH = '/home/watts/lal/Kaggle/kagglehp/scripts_nbs'
backup_project_as_zip(PROJECT_PATH, '../cache/code.scripts_nbs.%s.zip'%now)

2018-11-19 12:33:39.384598


In [28]:
# %%time
# !kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub8i1.csv -m ""

100%|████████████████████████████████████████| 480k/480k [00:08<00:00, 51.3kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationCPU times: user 332 ms, sys: 134 ms, total: 467 ms
Wall time: 15.6 s


In [29]:
# from time import sleep
# sleep(60)
# !kaggle competitions submissions -c human-protein-atlas-image-classification

fileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub8i.csv     2018-11-14 04:28:35               complete  0.443        None          
sub8h.csv     2018-11-13 23:48:00               complete  0.434        None          
sub8g.csv     2018-11-13 07:17:59               complete  0.389        None          
sub8c.csv     2018-11-11 14:31:02               complete  0.429        None          
sub30.csv     2018-11-09 07:02:56               complete  0.033        None          
sub29.csv     2018-11-08 22:07:11               complete  0.389        None          
sub28-c.csv   2018-11-08 15:47:08               complete  0.457        None          
sub28-bb.csv  2018-11-08 15:46:13               complete  0.458        None          
sub28-b.csv   2018-11-08 15:45:28               complete  0.454        None          
sub28-a.csv   2018-11-08 15:44:27               comple

In [26]:
from time import sleep
sleep(60)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName  date                 description  status    publicScore  privateScore  
--------  -------------------  -----------  --------  -----------  ------------  
sub8.csv  2018-10-20 20:08:45               complete  0.422        None          
sub7.csv  2018-10-20 17:06:09               complete  0.389        None          
sub5.csv  2018-10-19 18:27:33               complete  0.387        None          
sub4.csv  2018-10-19 14:45:15               complete  0.411        None          
sub3.csv  2018-10-19 10:19:26               complete  0.377        None          
sub2.csv  2018-10-19 08:07:30               complete  0.135        None          
sub1.csv  2018-10-19 06:28:57               complete  0.374        None          
